# Assignment 8

### 1.复习课上内容， 阅读相应论文。

### 2. 回答以下理论题目

#### 1.  What is autoencoder?

>在结构上，autoencoder是Feedforward Netural Network的一种，主要用于降维和特征抽取。
最简单的autoencoder是3层结构:Input Layer, Hidden Layer, Output Layer。其中Hidden Layer的维度远小于Input Layer，而Output用于重构Input，即目标是最小化$L(Input, Output)$。

>由于Hidden Layer很小，因此从Input到Hidden就是一个压缩的过程，也被称作encoder；从Hidden到Output的过程被称为decoder。由于autoencoder的目标是$f(x)=x$，因此如果模型loss很小，就说明Hidden层保留了大部分Input的信息，这也是autoencoder能用来作为特征抽取的主要原因。

>应用：图像降维，压缩，预训练，特征提取

#### 2. What are the differences between greedy search and beam search?

>这里以句子生成为例来解释。假设句子生成库只有3个词[a,b,c]。

>贪心搜索(greedy search):在生成每一个词时都选择概率最大的词。比如说生成第一个词时，[a,b,c]的概率分别为[0.1, 0.5, 0.4]，那么第一个词就是b，然后在此基础上计算第二个词的的概率，选择最大概率的那个词，以此类推直到终结符或句子最大长度。
存在的问题:虽然保证了在每次输出时都选择了最大概率的词，但是不能保证整个句子的概率是最大的，也就不一定能到达全局最优。比如说生成2个词的句子，安照贪心搜索，第一次选择了b，然后第二次的概率为[0.3,0.3,0.4]，那么句子为bc，累加概率为0.9，如果第一次选c(0.4)，可能第二次的概率会变成[0.8,0.1,0.1]，这样的话句子ca的累加概率为1.2，比贪心搜索的结果更好。

>集束搜索(beam search):和贪心搜索不同的是，集束搜索在选择的时候会保留概率最大的N个词，这个参数叫做beam size(或者beam width)。依旧是上面的例子，我们设定beam size=2：

>1. 在生成第一个词时，选择概率最大的2个词，假设是[a,b]
>2. 在生成第二个词的时候，以a和b为第一个词分别和库中所有词进行组合，得到新的序列[aa, ab, ac, ba, bb, bc]，然后从中选择概率最高的2个组合，假设为ac, ba
>3. 重复直到终结符或句子最大长度

>存在的问题:集束搜索其实也和贪心搜索一样，在N<词库大小时，也不一定能找到最优解。当词库很大时，如果N很大，会导致搜索空间太大而效率低下，所以一般会控制N而得到一个相对的较优解；当词库很小时，集束搜索则可以快速找到全局最优解。在这里，贪心搜索可以认为是beam size=1时的集束搜索特例

#### 3. What is the intuition of attention mechanism?

>attention机制的直观认识是：人在进行翻译的时候，不会在翻译某个词或者词组的时候把整个句子都看一遍，一般来说，都是通过关注这个词本身以及离这个词最近的一些词，然后再进行翻译。也就是说，我们会把焦点放在当前词所在的局部范围。而attention机制就是通过给予对当前输出位置比较重要(比较近)的输入位置较大的权重，较远的位置给予较小的权重来做到局部聚焦的作用。另外，由于attention是直接和input接触，所以也能做到长程联系。

>比如：看到I，那么自然可以翻译出我，看到swim，自然就可以翻译出游泳。因此可以把上文的词和下文的输出尝试直接联系起来。(而非仅仅依赖于latent vector)

>具体计算中，每一个时间步的输出$y^i$，会考虑三部分的信息：

1. $s^{i-1}$
2. $context^{i}$
3. $y^{i-1}$

>其中，$context = \sum_{i=1}^{T_x}{\alpha^{\langle t , t' \rangle}\cdot a^{t}}$

>a为encoder时间步的输出，$\alpha$为softmax(decoder前一时间步$s_{t-1}$与$a_{t}$的函数)

>$s_{t-1}$与$a_{t}$的函数有很多求法，其中三种：

1. $S_{t-1}\cdot w \cdot a_{t'}$
2. $S_{t-1}\cdot a_{t'}$ (outputs a value)
3. $V^T\cdot tanh(W_1 \cdot a_t + W_2 \cdot S_{t-1})$


#### 4. What is the disadvantage of word embeding introduced in previous lectures ?

>word embeding其中一个缺陷就是无法处理同义词。因为训练的时候是输入全部的语料库，输出的是每一个词的词向量，如果一个词有同义词，那么这个词的词向量其实是所有同义词的词向量混合。

#### 5. What is the architecture of ELMo model. (A brief description is enough)

>简单的说，ELMo模型先对语料库进行双层双向LSTM预训练，然后对训练完的ELMo网络输入新句子，句子中每个词都获得了对应的3层word embedding(底层word embedding，第一层LSTM embedding和第二层LSTM embedding)，通过学习到的权重加权累加得到最后的word embedding

#### 6. Compared to RNN,  what is the advantage of Transformer ?

>1. RNN每个输入的计算都依赖于前一个输入的输出，无法进行并行计算，而Transformer不存在这种序列依赖，所有输入能同步进行计算
2. RNN在建立长程联系的时候会出现bias或者梯度消失的问题，而Transformer可以通过self-attention让每个词和所有词进行直接交互，建立直接依赖，无论距离多远。

#### 7. Why we use layer normalizaiton instead of batch normalization in Transformer ?

>batch normalization其实是对一个Batch中每一个句子的同一位置的词的词向量进行normalization，但是事实上每个句子同一位置的词一般都不同，意思也不一样，这样进行normalization完全没有意义；而layer normalization是对每个词自己的词向量维度进行normalization，不存在这种问题。

#### 8. Why we need position embedding in Transformer ?

>因为Transformer里只有attention，而attention不像RNN会有一个时序的差异，attention对所有的(query, key)的计算是完全一样的，所以为了体现input的时序或位置的差异，需要额外输入一个position信息

#### 9. Briefly describe what is self-attention and what is multi-head attention?

>首先，attention模型分为3步，计算$Query$和$Key_i$的相似度，然后对第一步的输出进行softmax计算得到权重，最后权重$a_i$和对应的$Value_i$相乘得到对应的attention，其中一般情况下key=value

>* 自注意力模型(self-attention model): 是query=key=value的情况，例如输入一个句子，句子中的每一个词都要和该句子中所有的词进行attention计算，目的是学习句子内部词与词之间的关系。

>* 多头注意力模型(multi-head attention model):多头是对q，k，v进行N次(超参数)独立的线性变换，每次都算一个“头”，而且各自的参数W都不同，然后进行拼接后再进行一次线性变换得到多头attention的值

#### 10. What is the basic unit of GPT model?

>Masked Multi-head self-attention + Layer Norm + Feed Forward + Layer Norm

#### 11. Briefly descibe how to use GPT in other NLP tasks?

>* 对于分类问题(classification)，可以直接套用GPT模型: Input + Transformer + Linear + 分类层
>* 对于文本蕴含(Entailment)，需要对输入进行调整，即用分隔符(Delim)拼接前提p和假设h作为Input，加上Transformer和Linear
>* 文本相似度(Simlilarity)，把Input1和Input2用分隔符(Delim)按前后顺序拼接成Input1 Delim Input2 和 Input2 Delim Input1 然后分别进入Transformer后将结果相加，再加上Linear层
>* 多选题(Multiple Choice)，把Question分别和每一个Answer用分隔符拼接起来，输入到Transformer+Linear层，得到N个结果通过softmax层输出

#### 12. What is masked language model in BERT ?

>Mask language model是使用Transformer来训练language model的一种模型。

>输入词有80%几率被替换成【MASK】标签，10%几率随机替换成别的标签，10%几率不变。然后网络的目标就是预测【MASK】位置的词。

#### 13. What are the inputs of BERT ?

>BERT的输入为3个embedding的求和：

>* Token embedding表示当前词的embedding
>* Segment embedding表示所在句子的index embedding
>* Position embedding表示当前词所在位置的index embedding

>Word / Token Embeddings + Segment Embeddings + Position Embeddings

#### 14. Briely descibe how to use BERT in other NLP tasks.

>* 在分类问题中，只要在BERT输出层加个分类层就行
>* 在QA问题中，需要接受文本序列并且标注Answer，然后用BERT学习两个标注Answer开始和结束的向量来训练QA模型
>* 在命名实体识别(NER)中，需要对系统的文本中的各个实体进行标注(如人名，地点，时间)，然后将每个token的输出向量送到预测NER标签的分类层总

#### 15. What are the differences between these three models: GPT, BERT, GPT2.

>* GPT：结构为单向+Transformer，输入为Text + Pos embedding
>* BERT：结构为双向+Transformer，输入为word，segment + position embedding
>* GPT2：沿用GPT模型，但是放弃Fine-Tuning过程，用更大的数据集，更大的网络容量，并且调整网络结构，做到无监督训练的通用语言模型去完成不同的任务